In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        script = os.path.join(dirname, filename)
        with open(filename,'wb') as fw:
            with open(script,'rb') as fr:
                fw.writelines(fr.readlines())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spider-tool/evaluator.py
/kaggle/input/spider-tool/spider.py
/kaggle/input/spider-tool/extractor.py
/kaggle/input/corpus/docs2.json
/kaggle/input/corpus/docs1.json
/kaggle/input/corpus/lowdocs.json
/kaggle/input/albert/checkpoiot.pt
/kaggle/input/albert/base.pt


# 加载模型和分词器

In [2]:
import torch
from transformers import BertTokenizer, AlbertModel
from transformers import AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("uer/albert-large-chinese-cluecorpussmall")
# model = AlbertModel.from_pretrained("uer/albert-base-chinese-cluecorpussmall")
model = AutoModelForSequenceClassification.from_pretrained("uer/albert-large-chinese-cluecorpussmall", num_labels=2)
# model.load_state_dict(torch.load("/kaggle/input/albert/base.pt",map_location=torch.device('cpu')))

Some weights of the model checkpoint at uer/albert-large-chinese-cluecorpussmall were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at uer/albert-large-chinese-cluecorpussmall and are newly initializ

# 训练是否连续

# 爬取低质量文本

In [3]:
# from spider import Spider
# url='https://www.bilibili.com'
# spider = Spider('Bilibili_database/', 15, mode='inner')
# spider.add_url(url)
# spider.crawl()

# 

# 加载低质量文本

In [4]:
import json
lowtexts = []
with open('lowdocs.json') as f_obj:
    lowdocs = json.load(f_obj)       
    for site, docs_dict in lowdocs.items():
        for link, docs in docs_dict.items():
            lowtexts += docs

In [5]:
len(lowtexts)

4810

# 加载高赞文本

In [6]:


webtexts = []
with open('docs1.json') as f_obj:
    docs1 = json.load(f_obj)       
    for site, docs_dict in docs1.items():
        for link, docs in docs_dict.items():
            webtexts += docs
            
with open('docs2.json') as f_obj:
    docs2 = json.load(f_obj)       
    for site, docs_dict in docs2.items():
        for link, docs in docs_dict.items():
            webtexts += docs

In [7]:
np.random.shuffle(webtexts)
np.random.shuffle(lowtexts)
# print(webtexts[-1].split('\n'))
# print('-'*100)
# # print(lowtexts[-1].split('\n'))
# # webtexts

### 先判断两行是否连续,再判断连续行是否有意义

# 训练是否连续

In [8]:
def get_continous_text(text, pos=True):
    """
        在长文本text中采样;
        pos=True 取正样,返回连续的两行文本,pos=False取负样返回不连续的文本
        返回 text1(str),text2(str)
    """
    text = text.split('\n')
    max_len = len(text)
#     print(max_len)
    if  max_len <= 2:
        return '2. 排列：将所给对象随机排列','3. 分布：产生指定分布的数据，如高斯分布等'
    
    if pos:
        ind = np.random.randint(low=0,high=max_len-1)
        return text[ind][-32:],text[ind+1][:32]  # 一个中字占4个Token,这个切词器有点逆天
    else:
        ind = np.random.randint(low=0,high=max_len-2)
        ind2 = np.random.randint(low=ind+2,high=max_len)
        return text[ind][-32:],text[ind2][:32]

# 训练多行文本是否有意义

In [9]:
def get_threeline_text(text):
    text = text.split('\n')
    max_len = len(text)
#     print(max_len)
    if  max_len <= 3:
        return '\n'.join(text)
    
    
    ind = np.random.randint(low=0,high=max_len-3)
    return '\n'.join(text[ind:ind+3])  



# 为节省参数,进行混合训练

In [10]:
def get_batch(texts,lowtexts, batch_size=64, batch_num=1000):
    max_len = len(texts)
    while batch_num:
        batch_num -= 1
        text1_list = []
        text2_list = []
        label_list = []
        while(len(text1_list)<batch_size):
            ind = np.random.randint(low=0,high=max_len)
            text1,text2 = get_continous_text(texts[ind],pos=True)
            text1_list.append(text1)
            text2_list.append(text2)
            label_list.append(1)
            text1,text2 = get_continous_text(texts[ind],pos=False)
            text1_list.append(text1)
            text2_list.append(text2)
            label_list.append(0)
#         print("Len:",len(text1_list),"text1",text1_list)
#         print("Len:",len(text2_list),"text2",text2_list)
#         print("Len:",len(label_list),"label",label_list)
        yield tokenizer(text1_list,text2_list, truncation=True,max_length=128,padding=True ,return_tensors='pt'),torch.LongTensor(label_list)
    
        batch_num -= 1
        text1_list = []
        label_list = []
        while(len(text1_list)<batch_size):
            ind = np.random.randint(low=0,high=max_len)
            text1 = get_threeline_text(texts[ind])
            text1_list.append(text1)
            label_list.append(1)
            
            ind = np.random.randint(low=0,high=len(lowtexts))
            text1 = get_threeline_text(lowtexts[ind])
            text1_list.append(text1)
            label_list.append(0)
            
#         print("Len:",len(text1_list),"text1",text1_list)
#         print("Len:",len(label_list),"label",label_list)
        yield tokenizer(text1_list, truncation=True,max_length=128,padding=True ,return_tensors='pt'),torch.LongTensor(label_list)
  
    

In [11]:
import time
import numpy as np
def train(model,epoch):
    
    optimizer = torch.optim.Adam(model.parameters(),lr=3e-5,betas=(0.9,0.98),eps=1e-9)
    criterion = torch.nn.CrossEntropyLoss()
#     accumulation_steps = int(50 * np.sqrt(epoch))
    accumulation_steps = int(50 * epoch)
    steps = 0
    log_interval = 20
    total_loss = 0
    start_time = time.time()
    
    
    model.to(device)
    model.train()

    optimizer.zero_grad()
    
    batch_size = 64
    
    for encoded_input,label in get_batch(webtexts,lowtexts):

        encoded_input = encoded_input.to(device)
        label = label.to(device)
        output = model(**encoded_input,labels=label)
        
        pred = torch.argmax(output.logits,-1)

        loss = output.loss
        total_loss += loss.item()
        loss = loss/accumulation_steps        
        loss.backward()

        
        steps += 1
        if steps%accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        
        

        if steps%log_interval == 0:       
            
            lr = optimizer.param_groups[0]['lr']
            s_per_step = (time.time() - start_time) / log_interval
            cur_loss = total_loss / log_interval
            
            print(f'| steps {steps//accumulation_steps:5d}|'
                  f'lr {lr} | s/step {s_per_step:5.2f} | '
                  f'loss {cur_loss:5.2f} ')
            total_loss = 0
            start_time = time.time()
            
            
#             print("pred:",pred)
#             print("label:",label)
            acc =  np.mean((pred == label).tolist())
            print("acc:",acc)

            num1 = np.sum((1 == label).tolist())
            print("num1:",num1)
            if num1>0:    
                c1 = np.sum(((torch.argmax(output.logits,-1) == label) & (1 == label)).tolist())/np.sum((1 == label).tolist())
                print("correct of 1:",c1)
            
                c0 = np.sum(((torch.argmax(output.logits,-1) == label) & (0 == label)).tolist())/np.sum((0 == label).tolist())
                print("correct of 0:",c0)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for i in range(1,15):
    print("*"*60)
    print("epoch:",i)
    train(model,i)
    torch.save(model.state_dict(),"large.pt")
#     model.load_state_dict(torch.load("large.pt"))
torch.save(model.state_dict(),"large.pt")

************************************************************
epoch: 1
| steps     0|lr 3e-05 | s/step  1.94 | loss  0.71 
acc: 0.4375
num1: 32
correct of 1: 0.375
correct of 0: 0.5
| steps     0|lr 3e-05 | s/step  1.71 | loss  0.70 
acc: 0.53125
num1: 32
correct of 1: 0.46875
correct of 0: 0.59375
| steps     1|lr 3e-05 | s/step  1.70 | loss  0.69 
acc: 0.671875
num1: 32
correct of 1: 0.84375
correct of 0: 0.5
| steps     1|lr 3e-05 | s/step  1.70 | loss  0.67 
acc: 0.625
num1: 32
correct of 1: 0.875
correct of 0: 0.375
| steps     2|lr 3e-05 | s/step  1.71 | loss  0.67 
acc: 0.578125
num1: 32
correct of 1: 0.65625
correct of 0: 0.5
| steps     2|lr 3e-05 | s/step  1.71 | loss  0.64 
acc: 0.734375
num1: 32
correct of 1: 0.75
correct of 0: 0.71875
| steps     2|lr 3e-05 | s/step  1.71 | loss  0.64 
acc: 0.78125
num1: 32
correct of 1: 0.8125
correct of 0: 0.75
| steps     3|lr 3e-05 | s/step  1.71 | loss  0.63 
acc: 0.78125
num1: 32
correct of 1: 0.8125
correct of 0: 0.75
| steps     3|l

# 预测

In [13]:
model.load_state_dict(torch.load("large.pt"))

<All keys matched successfully>

In [14]:
def predict(model,text1,text2=None):
    model.eval()
    
    encoded_input = tokenizer(text1[-32:],text2[:32], padding=True ,return_tensors='pt').to(device) \
    if text2 else tokenizer(text1, padding=True ,return_tensors='pt').to(device)
    
    output = model(**encoded_input)
    prob = torch.softmax(output.logits,-1)
    p = prob[0][1].item()
#     print(p)
    model.train()
    return p

In [15]:
sent = """
明明是播放数最高的单曲之一，可还有萌新不知道这首歌;
明明在圈内外都算是元老级曲目了，可播放数收藏数还是涨势缓慢;
喜欢的人超多，可固定人气不高，传说遥遥无期，周刊榜上无名；
老曲就是这点不好~（￣▽￣）
（大家都在别的平台听过了所以没有来
原作者这里么~）(｀・ω・´)
"""

low = '''首页
番剧
电影
国创
电视剧
综艺
纪录片
动画
游戏
鬼畜
音乐
舞蹈
影视
娱乐
知识
科技
资讯
美食
生活
汽车
时尚
运动
动物圈
VLOG
搞笑
单机游戏
虚拟UP主
公益
公开课
专栏
直播
活动
课堂
社区中心
新歌热榜
番剧
直播
游戏中心
会员购
漫画
赛事
下载客户端'''

p = predict(model,"明明是播放数最高的单曲之一，可还有萌新不知道这首歌;","明明在圈内外都算是元老级曲目了，可播放数收藏数还是涨势缓慢;") 
print('正类  连续的概率:',p)
p = predict(model,"明明是播放数最高的单曲之一，可还有萌新不知道这首歌;","喜欢的人超多，可固定人气不高，传说遥遥无期，周刊榜上无名；") 
print('负类  连续的概率:',p)

p = predict(model,"明明是播放数最高的单曲之一，可还有萌新不知道这首歌;","获取我自己写的n篇UnderMine专栏中的166件圣物数据，将它们的效果、简评、评分按编号顺序录入已有的excel表格中，作为UM圣物查询软件的数据库。其中效果和简评需要保留html代码并修改成软件可渲染的html格式。") 
print('负类  连续的概率:',p)

p = predict(model,sent) 
print('正类  高赞的概率:',p)

p = predict(model,low)
print('负类  高赞的概率:',p)


正类  连续的概率: 0.742198646068573
负类  连续的概率: 0.6999433636665344
负类  连续的概率: 0.4058830142021179
正类  高赞的概率: 0.99174964427948
负类  高赞的概率: 0.13970160484313965
